# 📓 Text to Text Quickstart

In this quickstart you will create a simple text to text application and learn how to log it and get feedback.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/examples/quickstart/text2text_quickstart.ipynb)

## Setup
### Add API keys
For this quickstart you will need an OpenAI Key.

In [ ]:
!pip install trulens trulens-providers-openai openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

### Import from TruLens

In [ ]:
# Create openai client
from openai import OpenAI

# Imports main tools:
from trulens.core import Feedback
from trulens.core import TruSession
from trulens.providers.openai import OpenAI as fOpenAI

client = OpenAI()
session = TruSession()
session.reset_database()

### Create Simple Text to Text Application

This example uses a bare bones OpenAI LLM, and a non-LLM just for demonstration purposes.

In [ ]:
def llm_standalone(prompt):
    return (
        client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are a question and answer bot, and you answer super upbeat.",
                },
                {"role": "user", "content": prompt},
            ],
        )
        .choices[0]
        .message.content
    )

### Send your first request

In [ ]:
prompt_input = "How good is language AI?"
prompt_output = llm_standalone(prompt_input)
prompt_output

## Initialize Feedback Function(s)

In [ ]:
# Initialize OpenAI-based feedback function collection class:
fopenai = fOpenAI()

# Define a relevance function from openai
f_answer_relevance = Feedback(fopenai.relevance).on_input_output()

## Instrument the callable for logging with TruLens

In [ ]:
from trulens.apps.basic import TruBasicApp

tru_llm_standalone_recorder = TruBasicApp(
    llm_standalone, app_name="Happy Bot", feedbacks=[f_answer_relevance]
)

In [ ]:
with tru_llm_standalone_recorder as recording:
    tru_llm_standalone_recorder.app(prompt_input)

## Explore in a Dashboard

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session)  # open a local streamlit app to explore

# stop_dashboard(session) # stop if needed

## Or view results directly in your notebook

In [ ]:
session.get_records_and_feedback()[0]